# RoI adust setting

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from matplotlib.patches import Polygon

# 1. RoI 파일 로드
def load_roi(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 2. 특정 Site, Zone 및 Region 선택
def get_regions(roi_data, site, zone, regions=None):
    zone_data = roi_data[site][zone]
    structured_polygons = {}
    for region_name, region in zone_data.items():
        if regions is None or region_name in regions:
            structured_polygons[region_name] = np.array(region)
    return structured_polygons

# 3. 변환 함수 (이동, 회전, 크기 조정)
def transform_regions(regions, translation=(0,0), rotation=0, scale=1.0, rotation_center=None):
    transformed_regions = {}
    theta = np.radians(rotation)
    rot_matrix = np.array([[np.cos(theta), -np.sin(theta)], 
                            [np.sin(theta),  np.cos(theta)]])
    
    for region_name, region in regions.items():
        scaled_region = region * scale  # 1. 크기 변환

        if rotation_center is None:
            # 기본적으로 중심을 다각형의 평균 좌표로 설정
            cx, cy = np.mean(scaled_region, axis=0)
        else:
            # 사용자가 직접 중심점을 설정할 수도 있음
            cx, cy = rotation_center

        # 2. 중심점을 원점으로 이동
        centered_region = scaled_region - np.array([cx, cy])

        # 3. 회전 적용
        rotated_region = np.dot(centered_region, rot_matrix.T)

        # 4. 원래 위치로 이동
        translated_region = rotated_region + np.array([cx, cy])

        # 5. 최종 이동 변환 적용
        transformed_regions[region_name] = translated_region + np.array(translation)

    return transformed_regions

# 4. 결과 시각화
def plot_regions(base_regions, transformed_regions):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for region_name, base_region in base_regions.items():
        base_polygon = Polygon(base_region, closed=True, edgecolor='blue', facecolor='none', linewidth=2, label=f'Stand {region_name}')
        ax.add_patch(base_polygon)
    
    for region_name, transformed_region in transformed_regions.items():
        transformed_polygon = Polygon(transformed_region, closed=True, edgecolor='red', facecolor='none', linewidth=2, linestyle='dashed', label=f'Trans {region_name}')
        ax.add_patch(transformed_polygon)
    
    ax.set_xlim(min(min(region[:,0].min() for region in base_regions.values()), min(region[:,0].min() for region in transformed_regions.values())) - 50, 
                max(max(region[:,0].max() for region in base_regions.values()), max(region[:,0].max() for region in transformed_regions.values())) + 50)
    ax.set_ylim(min(min(region[:,1].min() for region in base_regions.values()), min(region[:,1].min() for region in transformed_regions.values())) - 50, 
                max(max(region[:,1].max() for region in base_regions.values()), max(region[:,1].max() for region in transformed_regions.values())) + 50)
    
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("RoI")
    ax.legend()
    ax.grid()
    plt.show()

# 5. 실행 예시
roi_file = './RoI.json'  # 파일 경로
data = load_roi(roi_file)

# 비교할 두 개의 사이트와 Zone 및 Region 선택 ['2', '3', '4', '5', '6']
site1, zone1, selected_regions1 = 'Site F', 'E', ['4']
site2, zone2, selected_regions2 = 'Site F_BEV', 'E', ['4']

base_regions = get_regions(data, site1, zone1, selected_regions1)
transformable_regions = get_regions(data, site2, zone2, selected_regions2)

# E4, E5, E6
translation = (-85, -39)  # x, y 이동
rotation_center = None
rotation = 2.1  # 시계 방향 회전 (도)
scale = 1.06 # 크기 확대 default : 1

transformed_regions = transform_regions(transformable_regions, translation, rotation, scale, rotation_center)

# 변환 전후 시각화
plot_regions(base_regions, transformed_regions)


# GeoAlign tranformation

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import re
import pandas as pd

# 1. RoI 파일 로드
def load_roi(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 2. 특정 Site의 모든 Zone 및 Region 선택
def get_all_regions(roi_data, site):
    site_data = roi_data.get(site, {})
    structured_polygons = {}
    
    for zone, zone_data in site_data.items():
        for region_name, region in zone_data.items():
            structured_polygons[(zone, region_name)] = np.array(region)
    
    return structured_polygons

def transform_regions(regions, transform_params):
    transformed_regions = {}
    
    for (zone, region_name), region in regions.items():
        # 해당 (Zone, Region)에 대한 변환값 가져오기 (없으면 기본값)
        params = transform_params.get((zone, region_name), {"translation": (0,0), "rotation": 0, "scale": 1.0, "rotation_center": None})
        translation = params["translation"]
        rotation = params["rotation"]
        scale = params["scale"]
        rotation_center = params["rotation_center"]
        
        theta = np.radians(rotation)
        rot_matrix = np.array([[np.cos(theta), -np.sin(theta)], 
                                [np.sin(theta),  np.cos(theta)]])
        
        scaled_region = region * scale  # 크기 변환

        if rotation_center is None:
            cx, cy = np.mean(scaled_region, axis=0)
        else:
            cx, cy = rotation_center

        centered_region = scaled_region - np.array([cx, cy])  # 중심점 이동
        rotated_region = np.dot(centered_region, rot_matrix.T)  # 회전 적용
        translated_region = rotated_region + np.array([cx, cy]) + np.array(translation)  # 최종 이동 적용

        transformed_regions[(zone, region_name)] = translated_region

    return transformed_regions

# 3. 차량 궤적 CSV 파일 로드
def load_trajectories_csv(file_path):
    df = pd.read_csv(file_path)

    # CSV 파일의 컬럼명 확인
    print("CSV 컬럼명:", df.columns)

    # 'lane' 컬럼이 있는지 확인
    if 'lane' not in df.columns:
        raise KeyError("❌ 'lane' 컬럼이 존재하지 않습니다. CSV 파일을 확인하세요.")

    # lane 컬럼에서 zone과 region_name 추출
    def extract_zone_region(lane_value):
        match = re.match(r"([a-zA-Z]+)(\d+)", lane_value)  # 알파벳 + 숫자 추출
        if match:
            return match.group(1), match.group(2)  # (zone, region_name)
        else:
            return lane_value, "0"  # 숫자가 없는 경우 기본값 ('crossroads', '0')

    df[['zone', 'region_name']] = df['lane'].apply(lambda x: pd.Series(extract_zone_region(x)))

    # 컬럼 변경 확인
    print("📢 변환된 zone-region 예시:")
    print(df[['lane', 'zone', 'region_name']].head())

    # track_id가 있는 경우, track_id, zone, region_name 기준으로 그룹별 순번 생성
    if 'track_id' in df.columns:
        df['group_index'] = df.groupby(['track_id', 'zone', 'region_name']).cumcount()
        # 그룹화 시에도 track_id를 포함하여 각 차량의 궤적을 개별적으로 관리
        group_keys = ['track_id', 'zone', 'region_name']
    else:
        df['group_index'] = df.groupby(['zone', 'region_name']).cumcount()
        group_keys = ['zone', 'region_name']

    # (track_id, zone, region_name) 기준으로 그룹화하여 리스트로 변환
    trajectories = {}
    for key, group in df.groupby(group_keys):
        trajectories[key] = group[['center_x', 'center_y']].values  # numpy array 변환

    return df, trajectories

# 4. 변환 함수 (RoI 및 차량 궤적에 동일하게 적용)
def transform_points(points, params):
    """ 단일 포인트(center_x, center_y) 또는 다수 포인트(np.array) 변환 """
    translation = params["translation"]
    rotation = params["rotation"]
    scale = params["scale"]
    rotation_center = params["rotation_center"]

    theta = np.radians(rotation)
    rot_matrix = np.array([[np.cos(theta), -np.sin(theta)], 
                           [np.sin(theta),  np.cos(theta)]])

    scaled_points = points * scale  # 크기 변환

    if rotation_center is None:
        cx, cy = np.mean(scaled_points, axis=0)
    else:
        cx, cy = rotation_center

    centered_points = scaled_points - np.array([cx, cy])  # 중심점 이동
    rotated_points = np.dot(centered_points, rot_matrix.T)  # 회전 적용
    transformed_points = rotated_points + np.array([cx, cy]) + np.array(translation)  # 최종 이동 적용

    return transformed_points

# 5. 차량 궤적 변환 (track_id 포함 그룹 기준)
def transform_trajectories(trajectories, transform_params):
    transformed_trajectories = {}
    # trajectories의 key는 (track_id, zone, region_name) 또는 (zone, region_name)
    for key, traj_points in trajectories.items():
        # key가 (track_id, zone, region_name)인 경우, 변환 파라미터는 (zone, region_name)로 조회
        if isinstance(key, tuple) and len(key) == 3:
            track_id, zone, region_name = key
        else:
            zone, region_name = key

        if (zone, region_name) in transform_params:
            params = transform_params[(zone, region_name)]
            transformed_trajectories[key] = transform_points(np.array(traj_points), params)
        else:
            transformed_trajectories[key] = np.array(traj_points)  # 변환 정보 없으면 그대로 유지

    return transformed_trajectories

# 6. 변형된 차량 궤적을 CSV로 저장 (track_id 포함 그룹 기준)
def save_transformed_trajectories(df, transformed_trajectories, output_file):
    """
    기존 데이터(df)에 변형된 center_x, center_y를 추가하여 저장 (메모리 최적화 버전)
    """
    # transformed_trajectories의 key는 (track_id, zone, region_name) 또는 (zone, region_name)
    transformed_dict = {
        key: transformed_points
        for key, transformed_points in transformed_trajectories.items()
    }

    def apply_transformation(row):
        # track_id가 있는 경우, 키에 포함
        if 'track_id' in row:
            key = (row['track_id'], row['zone'], row['region_name'])
        else:
            key = (row['zone'], row['region_name'])
            
        if key in transformed_dict:
            idx = row['group_index']  # 그룹 내 순번 사용
            arr = transformed_dict[key]
            # 안전장치: idx가 배열 범위를 벗어나면 마지막 좌표 사용
            if idx < len(arr):
                return pd.Series(arr[idx])
            else:
                return pd.Series(arr[-1])
        return pd.Series([row['center_x'], row['center_y']])  # 변환 정보가 없으면 기존 값 유지

    df[['transformed_center_x', 'transformed_center_y']] = df.apply(apply_transformation, axis=1)

    df.to_csv(output_file, index=False)
    print(f"✅ 변형된 차량 궤적이 저장되었습니다: {output_file}")

# 7. RoI 및 차량 궤적 시각화
def plot_all_regions_and_trajectories(base_regions, transformed_regions, base_trajectories, transformed_trajectories):
    fig, ax = plt.subplots(figsize=(12, 10))

    # 변형 전 RoI 및 차량 궤적 (파란색)
    for (zone, region_name), base_region in base_regions.items():
        base_polygon = Polygon(base_region, closed=True, edgecolor='blue', facecolor='none', linewidth=2)
        ax.add_patch(base_polygon)
        plt.text(np.mean(base_region[:,0]), np.mean(base_region[:,1]),
                 f"{zone}-{region_name}", fontsize=10, color='blue', ha='center')

    for key, base_traj in base_trajectories.items():
        ax.scatter(base_traj[:,0], base_traj[:,1], color='blue', s=10,
                   label='Original Trajectory' if key[1:] == ("A", "1") else "")

    # 변형된 RoI 및 차량 궤적 (빨간색)
    for (zone, region_name), transformed_region in transformed_regions.items():
        transformed_polygon = Polygon(transformed_region, closed=True, edgecolor='red',
                                      facecolor='none', linewidth=2, linestyle='dashed')
        ax.add_patch(transformed_polygon)
        plt.text(np.mean(transformed_region[:,0]), np.mean(transformed_region[:,1]),
                 f"{zone}-{region_name}", fontsize=10, color='red', ha='center')

    for key, transformed_traj in transformed_trajectories.items():
        # key가 (track_id, zone, region_name) 또는 (zone, region_name)
        if isinstance(key, tuple) and len(key) == 3:
            zone, region_name = key[1], key[2]
        else:
            zone, region_name = key
        ax.scatter(transformed_traj[:,0], transformed_traj[:,1], color='red', s=10,
                   label='Transformed Trajectory' if (zone, region_name) == ("A", "1") else "")

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("RoI")
    ax.legend()
    ax.grid()
    plt.show()

# 8. 실행 예시
roi_file = './RoI.json'  
traj_file = '/home/hdd2/drone/csv/G/lane/DJI_0101_stab.csv'  # 차량 궤적 CSV 파일
output_traj_file = '/home/hdd2/drone/code/trans/G/test/DJI_0101_stab_transformed.csv'  # 변환된 궤적 저장 경로

data = load_roi(roi_file)
df, trajectories = load_trajectories_csv(traj_file)

# Site 선택
site1 = 'Site G'
site2 = 'Site G_cut'

transform_params = {
    ("D", "1"): {"translation": (0, -35), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("D", "2"): {"translation": (0, -35), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("D", "3"): {"translation": (0, -27), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("D", "4"): {"translation": (0, -27), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("D", "5"): {"translation": (0, -27), "rotation": 2.4, "scale": 1.0, "rotation_center": None},

    ("B", "1"): {"translation": (0, -30), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("B", "2"): {"translation": (0, -30), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("B", "3"): {"translation": (0, -20), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("B", "4"): {"translation": (0, -20), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("B", "5"): {"translation": (0, -20), "rotation": 2.4, "scale": 1.0, "rotation_center": None},

    ("E", "1"): {"translation": (55, 22), "rotation": 2.6, "scale": 1.0, "rotation_center": None},
    ("E", "2"): {"translation": (0, 40), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("E", "3"): {"translation": (0, 40), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("E", "4"): {"translation": (0, 40), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("E", "5"): {"translation": (0, 40), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("E", "6"): {"translation": (0, 40), "rotation": 2.4, "scale": 1.0, "rotation_center": None},

    ("A", "1"): {"translation": (30, -85), "rotation": 2.7, "scale": 1.0, "rotation_center": None},
    ("A", "2"): {"translation": (760, 135), "rotation": 3.5, "scale": 0.8, "rotation_center": None},
    ("A", "3"): {"translation": (0, -60), "rotation": 2.5, "scale": 1.0, "rotation_center": None},
    ("A", "4"): {"translation": (0, -60), "rotation": 2.5, "scale": 1.0, "rotation_center": None},
    ("A", "5"): {"translation": (0, -45), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("A", "6"): {"translation": (0, -45), "rotation": 2.4, "scale": 1.0, "rotation_center": None},
    ("A", "7"): {"translation": (0, -45), "rotation": 2.4, "scale": 1.0, "rotation_center": None},

    ("C", "1"): {"translation": (0, 160), "rotation": 0.0, "scale": 0.8, "rotation_center": None},
    ("C", "2"): {"translation": (0, 160), "rotation": 0.0, "scale": 0.8, "rotation_center": None},
    ("C", "3"): {"translation": (0, 60), "rotation": 0.0, "scale": 0.9, "rotation_center": None},
    ("C", "4"): {"translation": (0, 60), "rotation": 0.0, "scale": 0.9, "rotation_center": None},
    ("C", "5"): {"translation": (0, -55), "rotation": 0.0, "scale": 1.0, "rotation_center": None},

    ("crossroads", "1"): {"translation": (-28, -125), "rotation": 0, "scale": 1.059, "rotation_center": None},
    ("crossroads", "2"): {"translation": (-50, -60), "rotation": 1.5, "scale": 1.05, "rotation_center": None}
}

# 모든 Zone 및 Region 가져오기
base_regions = get_all_regions(data, site1)
transformable_regions = get_all_regions(data, site2)

# RoI 변환 적용
transformed_regions = transform_regions(transformable_regions, transform_params)

# 차량 궤적 변환 적용
transformed_trajectories = transform_trajectories(trajectories, transform_params)

# 변형된 궤적을 CSV로 저장
save_transformed_trajectories(df, transformed_trajectories, output_traj_file)

# 변환 전후 모든 RoI 및 차량 궤적 시각화
plot_all_regions_and_trajectories(base_regions, transformed_regions, trajectories, transformed_trajectories)